# Install Packages

In [ ]:
!ls ../input/global-wheat-detection/test

In [ ]:
!pip install '../input/detectron2-wheat/pycocotools-2.0.0-cp37-cp37m-linux_x86_64.whl'
!pip install '../input/detectron2-wheat/yacs-0.1.7-py3-none-any.whl'
!pip install '../input/detectron2-wheat/fvcore-0.1.1.post200513-py3-none-any.whl'
!pip install '../input/detectron2-wheat/detectron2-0.1.2cu101-cp37-cp37m-linux_x86_64.whl'

In [ ]:
# import common utilities
from contextlib import contextmanager
import cv2
import glob
from itertools import groupby
import json
import matplotlib.pyplot as plt
import random
import numpy as np
import os
import pandas as pd
import re
import sys

In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
from detectron2.structures import BoxMode
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.data import MetadataCatalog, DatasetCatalog

# Prepare Kaggle Wheat Competition Dataset

In [ ]:
i = glob.glob('/kaggle/input/global-wheat-detection/train/*.jpg')
print('number of train images: {}'.format(len(i)))
# print('train images: {}'.format(i))

i = glob.glob('/kaggle/input/global-wheat-detection/test/*.jpg')
print('number of test images: {}'.format(len(i)))
# print('test images: {}'.format(i))

In [ ]:
df = pd.read_csv('/kaggle/input/global-wheat-detection/train.csv')
# print(df)

In [ ]:
wheat_list = df.to_dict('records')
print('number of label records: {}'.format(len(wheat_list)))

In [ ]:
images = [wheat['image_id'] for wheat in wheat_list]
images = list(dict.fromkeys(images))
print('number of wheat images: {}'.format(len(images)))

image_name2id = {}
for i, img in enumerate(images):
    # print(i, img)
    image_name2id[img] = i + 1
# print('Image Name to ID mapping: {}'.format(image_name2id))

image_id2name = {}
for i, img in enumerate(images):
    # print(i, img)
    image_id2name[i+1] = img
# print('Classes ID to Name mapping: {}'.format(image_id2name))

In [ ]:
wheat_classes = [wheat['source'] for wheat in wheat_list]
wheat_classes = list(dict.fromkeys(wheat_classes))
wheat_classes.sort()
print('wheat classes: {}'.format(wheat_classes))

num_classes = len(wheat_classes)
print('num of classes: {}'.format(num_classes))

class_name2id = {}
for i, c in enumerate(wheat_classes):
    # print(i, c)
    class_name2id[c] = i + 1
print('Classes Name to ID mapping: {}'.format(class_name2id))

class_id2name = {}
for i, c in enumerate(wheat_classes):
    # print(i, c)
    class_id2name[i+1] = c
print('Classes ID to Name mapping: {}'.format(class_id2name))

In [ ]:
wheat_dicts = {}
for idx, wheat in enumerate(wheat_list):
    if not wheat['image_id'] in wheat_dicts:
        image_id = wheat['image_id']
        wheat_dicts[image_id] = {}
        wheat_dicts[image_id]['file_name'] = os.path.join('./train', wheat['image_id']+'.jpg')
        wheat_dicts[image_id]['image_id'] = image_name2id[wheat['image_id']]
        wheat_dicts[image_id]['height'] = wheat['height']
        wheat_dicts[image_id]['width'] = wheat['width']
        wheat_dicts[image_id]['annotations'] = []
        ann = {}
        regex = re.compile(r'\[(\d*\.?\d*), (\d*\.?\d*), (\d*\.?\d*), (\d*\.?\d*)\]')
        match = regex.search(wheat['bbox'])
        ann['bbox'] = [float(match.group(1)), float(match.group(2)), float(match.group(3)), float(match.group(4))]
        ann['bbox_mode'] = BoxMode.XYWH_ABS
        ann['segmentation'] = []
        ann['category_id'] = class_name2id[wheat['source']]
        ann['id'] = idx + 1
        wheat_dicts[image_id]['annotations'].append(ann)
    else:
        ann = {}
        regex = re.compile(r'\[(\d*\.?\d*), (\d*\.?\d*), (\d*\.?\d*), (\d*\.?\d*)\]')
        match = regex.search(wheat['bbox'])
        ann['bbox'] = [float(match.group(1)), float(match.group(2)), float(match.group(3)), float(match.group(4))]
        ann['bbox_mode'] = BoxMode.XYWH_ABS
        ann['segmentation'] = []
        ann['category_id'] = class_name2id[wheat['source']]
        ann['id'] = idx + 1
        wheat_dicts[image_id]['annotations'].append(ann)

print(len(wheat_dicts))

In [ ]:
temp_dir = './temp'
os.makedirs(temp_dir, exist_ok=True)

In [ ]:
# if your dataset is in COCO format, this cell can be replaced by the following three lines:
# from detectron2.data.datasets import register_coco_instances
# register_coco_instances("my_dataset_train", {}, "json_annotation_train.json", "path/to/image/dir")
# register_coco_instances("my_dataset_val", {}, "json_annotation_val.json", "path/to/image/dir")

def get_wheat_dicts(wheat_dicts, mode='train'):

    dataset_dicts = []

    train_valid_split = 5
    for idx, wheat in enumerate(wheat_dicts):
        if mode == 'train' and (idx % train_valid_split) == 0:
            continue
        if mode == 'valid' and not (idx % train_valid_split) == 0:
            continue

        dataset_dicts.append(wheat_dicts[wheat])

    print('mode: {}, number of samples: {}'.format(mode, len(dataset_dicts)))
    return dataset_dicts

In [ ]:
train_dataset_dicts = get_wheat_dicts(wheat_dicts, 'train')
# print(train_dataset_dicts[0:10])

train_dataset_coco_dicts = {}
train_dataset_coco_dicts['annotations'] = [{'segmentation':[
                        [ann['bbox'][0], ann['bbox'][1],
                        ann['bbox'][0]+ann['bbox'][2], ann['bbox'][1],
                        ann['bbox'][0]+ann['bbox'][2], ann['bbox'][1]+ann['bbox'][3],
                        ann['bbox'][0], ann['bbox'][1]+ann['bbox'][3]]
           ],
           'area':ann['bbox'][2]*ann['bbox'][3],
           'iscrowd':0,
           'image_id':img['image_id'],
           'bbox':ann['bbox'],
           'category_id':ann['category_id'],
           'id': ann['id']}
           for img in train_dataset_dicts
           for ann in img['annotations']]
train_dataset_coco_dicts['images'] = [{'file_name':img['file_name'], 'id':img['image_id'], 'height':img['height'], 'width':img['width']} for img in train_dataset_dicts]
train_dataset_coco_dicts['categories'] = [{'supercategory':c, 'name':c, 'id':class_name2id[c]} for c in wheat_classes]

with open(os.path.join(temp_dir, 'wheat_train_coco.json'), 'w') as outfile:
    json.dump(train_dataset_coco_dicts, outfile)
outfile.close()

In [ ]:
valid_dataset_dicts = get_wheat_dicts(wheat_dicts, 'valid')
# print(valid_dataset_dicts[0:10])

valid_dataset_coco_dicts = {}
valid_dataset_coco_dicts['annotations'] = [{'segmentation':[
                        [ann['bbox'][0], ann['bbox'][1],
                        ann['bbox'][0]+ann['bbox'][2], ann['bbox'][1],
                        ann['bbox'][0]+ann['bbox'][2], ann['bbox'][1]+ann['bbox'][3],
                        ann['bbox'][0], ann['bbox'][1]+ann['bbox'][3]]
           ],
           'area':ann['bbox'][2]*ann['bbox'][3],
           'iscrowd':0,
           'image_id':img['image_id'],
           'bbox':ann['bbox'],
           'category_id':ann['category_id'],
           'id': ann['id']}
           for img in valid_dataset_dicts
           for ann in img['annotations']]
valid_dataset_coco_dicts['images'] = [{'file_name':img['file_name'], 'id':img['image_id'], 'height':img['height'], 'width':img['width']} for img in valid_dataset_dicts]
valid_dataset_coco_dicts['categories'] = [{'supercategory':c, 'name':c, 'id':class_name2id[c]} for c in wheat_classes]

with open(os.path.join(temp_dir, 'wheat_valid_coco.json'), 'w') as outfile:
    json.dump(valid_dataset_coco_dicts, outfile)
outfile.close()

In [ ]:
DatasetCatalog.clear()
from detectron2.data.datasets import register_coco_instances

register_coco_instances('wheat_train', {}, 'wheat_train_coco.json', temp_dir)
register_coco_instances('wheat_valid', {}, 'wheat_valid_coco.json', temp_dir)

In [ ]:
!rm ./temp/wheat_train_coco.json
!rm ./temp/wheat_valid_coco.json

# Prepare Config & Model

In [ ]:
model_selections = {}
model_selections = {
    1: {'depth': 50, 'num_classes': 7, 'model': '../input/detectron2-model/model_depth50_0299999.pth'},
    2: {'depth': 101, 'num_classes': 7,  'model': '../input/detectron2-model/model_depth101_0269999.pth'},
    3: {'depth': 101, 'num_classes': 1,  'model': '../input/detectron2-model/model_101_preod_1c_0026999.pth'},
    4: {'depth': 101, 'num_classes': 7,  'model': '../input/detectron2-model/model_101_preod_7c_0096999.pth'},
    5: {'depth': 101, 'num_classes': 1,  'model': '../input/detectron2-model/depth101_pre_1c_od_0086999.pth'},
    6: {'depth': 101, 'num_classes': 1,  'model': '../input/detectron2-model/depth101_pre_1c_od_0205999.pth'},
    7: {'depth': 101, 'num_classes': 1,  'model': '../input/detectron2-model/depth101_pre_1c_od_AdamP_0056999.pth'},
    60: {'depth': 101, 'num_classes': 1,  'model': '../input/detectron2-model/model_101_preod_1c_0026999_plr0.pth'},
    61: {'depth': 101, 'num_classes': 1,  'model': '../input/detectron2-model/model_101_preod_1c_0026999_plr1.pth'},
    62: {'depth': 101, 'num_classes': 1,  'model': '../input/detectron2-model/model_101_preod_1c_0026999_plr2.pth'},
    63: {'depth': 101, 'num_classes': 1,  'model': '../input/detectron2-model/model_101_preod_1c_0026999_plr3.pth'},
    64: {'depth': 101, 'num_classes': 1,  'model': '../input/detectron2-model/model_101_preod_1c_0026999_plr4.pth'},
    65: {'depth': 101, 'num_classes': 1,  'model': '../input/detectron2-model/model_101_preod_1c_0026999_plr5.pth'},
    66: {'depth': 101, 'num_classes': 1,  'model': '../input/detectron2-model/model_101_preod_1c_0026999_plr6.pth'},
    67: {'depth': 101, 'num_classes': 1,  'model': '../input/detectron2-model/model_101_preod_1c_0026999_plr7.pth'},
    68: {'depth': 101, 'num_classes': 1,  'model': '../input/detectron2-model/model_101_preod_1c_0026999_plr8.pth'},
    69: {'depth': 101, 'num_classes': 1,  'model': '../input/detectron2-model/model_101_preod_1c_0026999_plr9.pth'},
    601: {'depth': 101, 'num_classes': 1,  'model': '../input/detectron2-model/model_101_preod_1c_0026999_plall_r1.pth'},
    610: {'depth': 101, 'num_classes': 1,  'model': '../input/detectron2-model/model_101_preod_1c_0026999_plall_r10.pth'},
    }
model_sel = model_selections[7]
print(model_sel)

In [ ]:
cfg = get_cfg()
# cfg.OUTPUT_DIR = '/content/drive/MyDrive/NCTU/基於深度學習之視覺辨識專論/HW/Final/output1'
# cfg.MODEL.DEVICE = 'cpu'
cfg.OUTPUT_DIR = './'
# cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.merge_from_file('../input/detectron2-base-yaml/BASE.yaml')

cfg.MODEL.MASK_ON = True
cfg.MODEL.RESNETS.DEPTH = model_sel['depth']
cfg.MODEL.WEIGHTS = model_sel['model']  # path to the model we just trained

cfg.DATASETS.TRAIN = ('wheat_train',)
cfg.DATASETS.TEST = ('wheat_valid',)
cfg.TEST.EVAL_PERIOD = 2000
cfg.SOLVER.CHECKPOINT_PERIOD = 1000
cfg.DATALOADER.NUM_WORKERS = 2
# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.0000025  # 0.00025: 0~12999; 0.0001: 13000~
cfg.SOLVER.MAX_ITER = 300000  # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.GAMMA = 0.5
# cfg.SOLVER.STEPS = (60000,100000,210000,250000) 0.5 gamma
cfg.SOLVER.STEPS = (30000,)
# cfg.SOLVER.LR_SCHEDULER_NAME = WarmupMultiStepLR
cfg.MODEL.SEM_SEG_HEAD.NUM_CLASSES = 0
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512
cfg.MODEL.ROI_HEADS.NUM_CLASSES = model_sel['num_classes']  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)

In [ ]:
print(cfg)

In [ ]:
# 0.7=55, 0.3=60, 0.1=57, 0=26
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.2   # set a custom testing threshold
cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.45

predictor = DefaultPredictor(cfg)

# Predict

In [ ]:
def bbox_adjust_ddd(bbox, input_format, output_format, adjust_momde, adjust_value):
    
    bbox = bbox.tolist()
    if input_format == 'XYXY':
        xmin = round(bbox[0])
        ymin = round(bbox[1])
        width = round(bbox[2] - bbox[0])
        height = round(bbox[3] - bbox[1])
    elif input_format == 'XYWH':
        xmin = round(bbox[0])
        ymin = round(bbox[1])
        width = round(bbox[2])
        height = round(bbox[3])
    
    if adjust_momde == 'PERCENTAGE':
        xmin += round(width * adjust_value)
        ymin += round(height * adjust_value)
        width -= round(width * adjust_value * 2)
        height -= round(height * adjust_value * 2)
    elif adjust_momde == 'PIXEL':
        xmin += adjust_value
        ymin += adjust_value
        width -= adjust_value * 2
        height -= adjust_value * 2
    elif adjust_momde == 'SHIFT':
        xmin += adjust_value
        ymin += adjust_value

    if output_format == 'XYXY':
        return [xmin, ymin, xmin+width, ymin+height]
    elif output_format == 'XYWH':
        return [xmin, ymin, width, height]

In [ ]:
def bbox_adjust(bbox, input_format, output_format, adjust_momde, adjust_value):
    
    bbox = bbox.tolist()
    if input_format == 'XYXY':
        xmin = (bbox[0])
        ymin = (bbox[1])
        width = (bbox[2] - bbox[0])
        height = (bbox[3] - bbox[1])
    elif input_format == 'XYWH':
        xmin = (bbox[0])
        ymin = (bbox[1])
        width = (bbox[2])
        height = (bbox[3])
    
    if adjust_momde == 'PERCENTAGE':
        xmin += (width * adjust_value)
        ymin += (height * adjust_value)
        width -= (width * adjust_value * 2)
        height -= (height * adjust_value * 2)
    elif adjust_momde == 'PIXEL':
        xmin += adjust_value
        ymin += adjust_value
        width -= adjust_value * 2
        height -= adjust_value * 2
    elif adjust_momde == 'SHIFT':
        xmin += adjust_value
        ymin += adjust_value

    if output_format == 'XYXY':
        return [xmin, ymin, xmin+width, ymin+height]
    elif output_format == 'XYWH':
        return [xmin, ymin, width, height]

In [ ]:
testfiles = [f for f in os.listdir('/kaggle/input/global-wheat-detection/test') if os.path.isfile(os.path.join('/kaggle/input/global-wheat-detection/test', f))]
# print(testfiles)

filenames = []
predicts = []

for idx, f in enumerate(testfiles):
    # print('image filename: {}'.format(f))
    print('{} {}'.format(idx, os.path.join('/kaggle/input/global-wheat-detection/test', f)))
    image = cv2.imread(os.path.join('/kaggle/input/global-wheat-detection/test', f))

    outputs = predictor(image)

    result = {}

    # result['image_id'] = f['id']
    result['image_id'] = os.path.splitext(f)[0]
    # result['score'] = [[round(i.tolist(),2)] for i in outputs['instances'].scores]
    result['score'] = [[0.9] for i in outputs['instances'].scores]
    result['category_id'] = [cate_id.numpy()+1 for cate_id in outputs["instances"].pred_classes.to('cpu')]
    result['bbox'] = [bbox_adjust(bbox, 'XYXY', 'XYWH', '', 0) for bbox in outputs["instances"].pred_boxes]
    # bbox_adjust(bbox, 'XYXY', 'XYWH', 'PERCENTAGE', 0.05)
    print(result)
    
    pred = []
    n_instances = len(outputs['instances'].scores)
    print(n_instances)
    for i in range(n_instances):  # Loop all instances
        # save information of the instance in a dictionary then append on coco_dt list
        pred.extend(result['score'][i])
        pred.extend(result['bbox'][i])

    filenames.append(result['image_id'])
    predicts.append(pred)
    print(pred)

print('filename list: {}'.format(filenames))
# print('predict list: {}'.format(predicts))
predicts = [' '.join(str(p) for p in pred) for pred in predicts]
print('predict list: {}'.format(predicts))

submission_dict = {
    'image_id': filenames,
    'PredictionString': predicts
}
submission = pd.DataFrame(submission_dict)
# submission.to_csv(os.path.join(cfg.OUTPUT_DIR, 'submission.csv'), encoding = 'utf-8',index = False)

# submission.to_json(os.path.join('./', 'submission.json'), index = False)

In [ ]:
submission.head

In [ ]:
submission.to_csv('submission.csv', index = False)

In [ ]:
# !cp ../input/test-submission/submission.csv ./